# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [605]:
# import libraries

import numpy as np
import pandas as pd
import nltk
import sqlite3
import sqlalchemy
import sys
import io
import warnings

from typing import List, Tuple

from sqlalchemy import create_engine


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, fbeta_score
from sklearn.exceptions import UndefinedMetricWarning


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [357]:
# Suppress the specific UserWarning(tokenizer in NLP vectorizer)
#warnings.filterwarnings("ignore", message="The parameter 'token_pattern' will not be used since 'tokenizer' is not None'")
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [359]:
# Connect to known databases
conn_chrismoetable = sqlite3.connect('chrismoetable.db')
conn_disasterresponse = sqlite3.connect('DisasterResponse.db')
conn_insertdb = sqlite3.connect('InsertDatabaseName.db')

def list_tables(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return connection.execute(query).fetchall()

# List tables for each database
chrismoetable_tables = list_tables(conn_chrismoetable)
disasterresponse_tables = list_tables(conn_disasterresponse)
insertdatabase_tables = list_tables(conn_insertdb)

# Close the connections
conn_chrismoetable.close()
conn_disasterresponse.close()
conn_insertdb.close()

# Output the results
print(chrismoetable_tables)
print(disasterresponse_tables)
print(insertdatabase_tables)

[]
[('Message',)]
[('process_data_table',), ('final',), ('chrismo',)]


In [361]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('chrismo', con=engine)

In [365]:
df.head(2)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [367]:
#reviewing data's completeness
df.isna().sum().sum()

0

In [369]:
df.nunique()

id                        26180
message                   26177
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_related               2
floods  

In [371]:
#Spliting data

X = df['message']
Y = df.iloc[:,4:]

In [373]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [375]:
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [377]:
print(X.shape)
print(Y.shape)

(26216,)
(26216, 34)


### 2. Write a tokenization function to process your text data

In [384]:
def tokenize(text: str)-> List[str]:
    """
    Tokenize and process the input text.

    This function performs tokenization, stopword removal, part o speech tagging, and lemmatization on the input text.

    Args:
        text (str): The input text to be processed.

    Returns:
        list: A list of processed tokens.
    """
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    STOPWORDS = stopwords.words("english")
    word_tokens = [word for word in word_tokens if word not in STOPWORDS]   #stop word removal
    
    tokens = []
    for word_token in word_tokens:
        token = lemmatizer.lemmatize(word_token.lower().strip(), pos='v')        #pos tagging
        tokens.append(token)
        
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

`pipeline = Pipeline([
                    ('features', FeatureUnion([
                        ('text_pipeline', Pipeline([
                            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                            ('tfidf_transformer', TfidfTransformer())
                        ])),
                    ])),
                    ('classifier', MultiOutputClassifier())
                ])`

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# Split the data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size = 0.25)

Testing a list of BASE models 

In [399]:
kn = KNeighborsClassifier()
naive = MultinomialNB()
random_forest = RandomForestClassifier(random_state=42)
xg = XGBClassifier(random_state=42)
dec_tree = DecisionTreeClassifier(random_state=42)

In [401]:
models = [kn, naive, random_forest, xg, dec_tree]

In [403]:
def train_predict() -> List[Tuple[str, float, float]]:
    """
    Feeds BASE models from our list into training pipelines, predicts using training and test data to detetc over or under-fitting
    
    Args: 
        None
        
    Returns: 
        List[Tuple[str, float, float]]: A list of tuples containing model name, test accuracy, and train accuracy.
    """
    results = []
    for model in models:
        pipeline = Pipeline([
            ('features', FeatureUnion([
                ('text_pipeline', TfidfVectorizer(tokenizer=tokenize))
            ])),
            ('classifier', MultiOutputClassifier(model))
        ])
        model_name = model.__class__.__name__
        print(model_name)
        pipeline.fit(X_train, y_train)
        test_accuracy = accuracy_score(y_test, pipeline.predict(X_test))
        print(f'Test Accuracy = {test_accuracy}')
        train_accuracy = accuracy_score(y_train, pipeline.predict(X_train))
        print(f'Train Accuracy = {train_accuracy}')
        print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
        results.append((model_name, test_accuracy, train_accuracy))
    
    return results

In [288]:
train_predict()

KNeighborsClassifier
Test Accuracy = 0.42523649679584985
Train Accuracy = 0.4837249516834503
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
MultinomialNB
Test Accuracy = 0.3642050656087885
Train Accuracy = 0.3977723527616723
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
RandomForestClassifier
Test Accuracy = 0.4104363747329875
Train Accuracy = 0.9971518665446037
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[('KNeighborsClassifier', 0.42523649679584985, 0.4837249516834503),
 ('MultinomialNB', 0.3642050656087885, 0.3977723527616723),
 ('RandomForestClassifier', 0.4104363747329875, 0.9971518665446037),
 ('XGBClassifier', 0.4247787610619469, 0.6004984233546944),
 ('DecisionTreeClassifier', 0.2551113823619164, 0.9979656189604312)]

Although accuracy is not our main metric here, it is a good indicator for what models to use.

From our results, all our models underfit. You can use this [reference](https://aws.amazon.com/what-is/overfitting/) to understand these concepts. 

To put into context about our base models, 

* `KNeighborsClassifier` gives a very descent result out the box, the best values even though it underfits.
* `MultinomialNB`: This Naive algorithm performs weakest using our metric here.
* `RandomForestClassifier`: Random Forest gives a descent result, but it overfits quite largely.
* `XGBClassifier`: Decent accuracy, only topped by our neighbors classifier. Overfits slightly less compared to other ensemble tree methods.
* `DecisionTreeClassifier`: Doesnt do well, as we expect really.


Moving forward, we would use XGBoost due to its hyperparameter tuning capabilities being so flexible and its performance out of the box.

In [407]:
X_train.shape

(19662,)

In [409]:
y_train.shape

(19662, 34)

Using `soft_prob` objective function means we try to obtain the probability of that input being in that exact class. It is the same logic used [here](https://forecastegy.com/posts/xgboost-multiclass-classification-python/). Now we reintroduce our training `Pipeline`

In [540]:
pipeline = Pipeline([
                    ('features', FeatureUnion([
                        ('text_pipeline', Pipeline([
                            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                            ('tfidf_transformer', TfidfTransformer())
                        ])),
                    ])),
                    ('classifier', MultiOutputClassifier(XGBClassifier(
                        max_depth=6,
                        learning_rate=0.1,
                        n_estimators=100,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        eval_metric=['mlogloss', 'merror']
                    )))
                ])

In [542]:
print(X_train.shape)
print(y_train.shape)

(19662,)
(19662, 34)


In [544]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x3a9828ea0>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())]))])),
                ('classifier',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               cols...
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=0.1,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=6,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...)))])

In [545]:
y_pred = pipeline.predict(X_test)

In [546]:
y_pred.shape

(6554, 34)

In [547]:
y_test.shape

(6554, 34)

In [594]:
accuracy_score(y_test, y_pred)

0.44003661885871226

In [596]:
accuracy_score(y_train, pipeline.predict(X_train))

0.5210049842335469

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [607]:
# Convert to DataFrame
y_test_df = pd.DataFrame(y_test, columns=y_test.columns)
predictions_df = pd.DataFrame(y_pred, columns=y_test_df.columns)

print("## Detailed Metrics Breakdown by Category")

overall_metrics = []

for column in y_test_df.columns:
    print(f"\n### {column}")
    report = classification_report(y_test_df[column], predictions_df[column], output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    class_labels = sorted(df_report.index.drop(['accuracy', 'macro avg', 'weighted avg']))
    df_report = df_report.loc[class_labels + ['weighted avg'], ['precision', 'recall', 'f1-score']].round(3)
    
    print(tabulate(df_report, headers='keys', tablefmt='pipe', showindex=True))
    
    weighted_avg = df_report.loc['weighted avg']
    overall_metrics.append({
        'Category': column,
        'Precision': weighted_avg['precision'],
        'Recall': weighted_avg['recall'],
        'F1-score': weighted_avg['f1-score']
    })

# Store overall_metrics for use in Part 2
overall_df = pd.DataFrame(overall_metrics).set_index('Category');

## Detailed Metrics Breakdown by Category

### request
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0.0          |       0.912 |    0.975 |      0.943 |
| 1.0          |       0.816 |    0.542 |      0.651 |
| weighted avg |       0.896 |    0.902 |      0.893 |

### offer
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0.0          |       0.995 |    1     |      0.997 |
| 1.0          |       0     |    0     |      0     |
| weighted avg |       0.99  |    0.995 |      0.992 |

### aid_related
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0.0          |       0.766 |    0.872 |      0.815 |
| 1.0          |       0.767 |    0.612 |      0.681 |
| weighted avg |       0.766 |    0.766 |      0.761 |

### medical_help
|              |   precision |   recall |   f1-score |
|:-------------|---

/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [609]:
print("\n## Overall Performance Summary")
print(tabulate(overall_df, headers='keys', tablefmt='pipe', showindex=True))

print("\n## Best and Worst Performing Categories")
best_category = overall_df['F1-score'].idxmax()
worst_category = overall_df['F1-score'].idxmin()
print(f"**Best:** {best_category} (F1-score: {overall_df.loc[best_category, 'F1-score']:.3f})")
print(f"**Worst:** {worst_category} (F1-score: {overall_df.loc[worst_category, 'F1-score']:.3f})")


## Overall Performance Summary
| Category               |   Precision |   Recall |   F1-score |
|:-----------------------|------------:|---------:|-----------:|
| request                |       0.896 |    0.902 |      0.893 |
| offer                  |       0.99  |    0.995 |      0.992 |
| aid_related            |       0.766 |    0.766 |      0.761 |
| medical_help           |       0.913 |    0.928 |      0.913 |
| medical_products       |       0.95  |    0.957 |      0.949 |
| search_and_rescue      |       0.974 |    0.978 |      0.972 |
| security               |       0.969 |    0.982 |      0.974 |
| military               |       0.965 |    0.972 |      0.966 |
| water                  |       0.964 |    0.966 |      0.965 |
| food                   |       0.955 |    0.956 |      0.955 |
| shelter                |       0.947 |    0.951 |      0.948 |
| clothing               |       0.988 |    0.989 |      0.988 |
| money                  |       0.978 |    0.982 |      0

### 6. Improve your model
Use grid search to find better parameters. 

In [647]:
param_gg = {
                'classifier__estimator__max_depth': [3, 6, 9],
                'classifier__estimator__learning_rate': [0.01, 0.1, 0.3],
                'classifier__estimator__n_estimators': [100, 200, 300],
                'classifier__estimator__subsample': [0.6, 0.8, 1.0],
                'classifier__estimator__colsample_bytree': [0.6, 0.8, 1.0],
    
              }

In [649]:
grid_search = GridSearchCV(
                            estimator=pipeline,
                            param_grid=param_gg,
                            cv=3,
                            n_jobs=1,
                            verbose=2
)

In [ ]:
# Fit the grid search
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


In [ ]:
# Best parameters
best_params = grid_search.best_params_
#preditcitng using best estimator 
print("Best parameters found: ", best_params)

In [ ]:
y_pred_grid = cv.best_estimator_.predict(X_test)

In [ ]:
Load ??? omo

hehehe

ok sha



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.